# Merging DataFrames in Pandas
This notebook demonstrates various ways to merge DataFrames in Pandas, including different join types and key configurations.

## Importing Libraries
We start by importing the necessary libraries.

In [ ]:
import pandas as pd

## Loading Datasets
Load the transactions and clients datasets to demonstrate merging operations.

In [ ]:
# Load transactions dataset
transactions = pd.read_csv("../data/transactions.csv")
transactions.head()

In [ ]:
# Load clients dataset
clients = pd.read_csv("../data/clients.csv")
clients.head()

## Merging Transactions and Clients
Perform a left join to combine transactions with client information based on the `idCliente` column.

In [ ]:
# Merge transactions with clients using a left join
transactions.merge(right=clients,
                   how="left",
                   on=["idCliente"],
                   suffixes=["Transacao", "Cliente"] # Add suffixes to differentiate overlapping columns
                             )
# Transactions -> left

## Creating Example DataFrames
Define two small DataFrames to demonstrate merging with different key names.

In [ ]:
# Define the first DataFrame
df_1 = pd.DataFrame(
    {
        "transacao": [1, 2, 3, 4, 5],
        "idCliente": [1, 2, 3, 2, 2],
        "valor": [10, 90, 30, 45, 60]
    }
)

In [ ]:
# Define the second DataFrame
df_2 = pd.DataFrame(
    {
       "id": [1, 2, 3, 4],
       "nome": ["marceline", "naniquinha", "mon cherri", "stacy"] 
    }
)

### Merging with Different Key Names
Perform a left join where the key columns have different names in the two DataFrames.

In [ ]:
# Merge df_1 and df_2 using different key names
df_1.merge(df_2, left_on=["idCliente"], right_on=["id"], how="left") # Merge with primary and foreign keys having different names

## Merging Transactions, Products, and Clients
Load additional datasets and demonstrate more complex merging operations.

In [ ]:
# Load transaction-product dataset
transaction_product = pd.read_csv("../data/transaction_product.csv")
transaction_product.head()

In [ ]:
# Load products dataset
products = pd.read_csv("../data/products.csv")
products.head()

### Combining Transactions and Products
Merge transactions with products through the transaction-product mapping.

In [ ]:
# Merge transactions with transaction-product mapping
client_transaction_product = transactions.merge(
    transaction_product, on="idTransacao",
    how="left"
)[["idTransacao", "idCliente", "idProduto"]]

client_transaction_product

In [ ]:
# Merge the result with products dataset
df_full = client_transaction_product.merge(
    products,
    on=["idProduto"]
)

### Filtering and Aggregating Data
Filter the merged dataset for a specific product and find the client with the highest number of transactions.

In [ ]:
# Filter for a specific product
df_full = df_full[df_full["descProduto"] == "Presença Streak"]

In [ ]:
# Group by client and count transactions
(df_full.groupby(by=["idCliente"])["idTransacao"]
    .count()
    .sort_values(ascending=False)
    .head(1))

### Optimized Approach
Demonstrate a more efficient way to achieve the same result using chained operations.

In [ ]:
# Optimized approach with chained operations

# Filter products
products = products[products["descProduto"] == "Presença Streak"]

# Perform merges and aggregation
(transactions.merge(transaction_product, on="idTransacao", how="left",
    ).merge(products, on=["idProduto"], how="right") # -> right join keeps only filtered values
    .groupby(by="idCliente")["idTransacao"]
    .count()
    .sort_values(ascending=False)
    .head(1)
)